In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely import affinity
import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
gdf = gpd.read_file('../data/na_lakes/hydrography_p_lakes_v2/hydrography_p_lakes_v2.shp')

In [ ]:
gdf.crs

In [ ]:
gdf = gdf.to_crs(epsg=3857)
gdf = gdf.dropna()

In [ ]:
gdf[gdf['NAMEEN'].str.contains('Cayuga')]

In [ ]:
lake_bounds = gdf[gdf['NAMEEN'].str.contains('Cayuga')]['geometry'].unary_union
# lake_bounds = affinity.scale(lake_bounds, xfact=1.5, yfact=1)
# lake_bounds = affinity.translate(lake_bounds, xoff=3000.0, yoff=0)
# lake_bounds = affinity.rotate(lake_bounds, xoff=3000.0, yoff=0)

In [ ]:
from pyproj import Transformer

def get_state_bounds(abbr):
    # get bounds for state and convert to epsg:3857
    usa = gpd.read_file('../data/states_21basic/states.shp')

    state_bounds = usa[usa.STATE_ABBR == abbr].total_bounds.reshape((2, 2))

    transformer = Transformer.from_crs("epsg:4326", "epsg:3857")

    state_bounds[0] = transformer.transform(state_bounds[0,1]-0.1, state_bounds[0,0]-0.1)
    state_bounds[1] = transformer.transform(state_bounds[1,1]+0.1, state_bounds[1,0]+0.1)
    
    return state_bounds

In [ ]:
# ax = gdf[gdf['NAMEEN'].str.contains('Seneca')].plot(
#     figsize=(18, 18), 
#     label='Seneca Lake',
# )

fig= plt.figure(figsize=(18, 18))

ax = plt.plot(*lake_bounds.exterior.xy)
ax = plt.gca()

ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=8)
ax.set_axis_off()

In [ ]:
nh_lakes = gpd.read_file('../data/na_lakes/NH Water Bodies/NH Water Bodies.shp')

In [ ]:
nh_lakes = nh_lakes.to_crs(epsg=3857)
nh_lakes = nh_lakes.dropna()

In [ ]:
nh_lakes.head()

In [ ]:
len(nh_lakes['GNIS_Name'].unique())

In [ ]:
sunapee_lake = nh_lakes[nh_lakes['GNIS_Name'] == 'Sunapee Lake']
sunapee_lake

In [ ]:
lake_bounds = sunapee_lake['geometry'].unary_union

In [ ]:
fig= plt.figure(figsize=(18, 18))

ax = plt.plot(*lake_bounds.exterior.xy)
ax = plt.gca()

ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=8)
ax.set_axis_off()